# Inference of Trained Retinanet model 
### Import Libraries

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy
import math

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

Using TensorFlow backend.


In [2]:
#file_location = '/home/paperspace/HPM/BachelorProject/02 Deep Learning/Create-CSV-dataset/list_of_img_in_val_set_12-03.csv' #'list_of_img_in_val_set4.csv'
file_location = '../../Create-CSV-dataset/list_of_img_in_val_set_12-03.csv'
place_to_store_results = 'Evaluations/'
path_img_folder = '../../../03 Data/Simple Dataset/'
accept_BB_threshold = 0.95

def draw_vector(img, x, y, box):
    angle = np.arctan2(y,x)
    angle = angle/2
    center_x = (box[0]+box[2])/2
    center_y = (box[1]+box[3])/2
    pt1_x = -25*np.cos(angle)+center_x
    pt1_y = -25*np.sin(angle)+center_y
    pt2_x = 25*np.cos(angle)+center_x
    pt2_y = 25*np.sin(angle)+center_y
    cv2.line(img, (int(pt1_x),int(pt1_y)), (int(pt2_x), int(pt2_y)), color = (0, 255,255), thickness = 2, lineType = 2)
    return
    

def convert_rect_to_point(rect):
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    
    return (x1, y1, x2, y2)

def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        annotation_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            annotation_list.append(convert_rect_to_point(rect))
    return annotation_list; 


def max_val(val1, val2):
    if val1 > val2:
        return val1
    else:
        return val2
    
def min_val(val1, val2):
    if val1 > val2:
        return val2
    else:
        return val1
    
    
def filter_bounding_boxes(bounding_boxes, scores):
    list_of_BB = []
    for box, score in zip(bounding_boxes, scores):
        if score < accept_BB_threshold: 
            break
        list_of_BB.append(box)
    return list_of_BB


def area(BB):
    width = abs(BB[0] - BB[2])
    height = abs(BB[1] - BB[3])
    return height * width

def intersection(BB1, BB2):
    # find coordiantes of intersection rectangle 
    (x1, y1, x2, y2) = BB1
    (x3, y3, x4, y4) = BB2
    if x1 > x4 or x3 > x2 or y3 > y2 or y1 > y4:
        return 0
    x5 = max_val(x1, x3);
    y5 = max_val(y1, y3);
    x6 = min_val(x2, x4);
    y6 = min_val(y2, y4);  
    BB = (x5, y5, x6, y6)
    # TODO: check if no intersection exists
    return area(BB)
    
def union(BB1, BB2): 
    return area(BB1) + area(BB2) - intersection(BB1, BB2)

def evaluate_predictions(annotations, preds, IOU_thres):
    predictions = copy.deepcopy(preds)
    false_neg = 0
    for annotation in annotations: 
        #annotation = convert_rect_to_point(anno)
        max_IOU = 0
        for i in range(len(predictions)): 
            #print('Annotation: ', annotation)
            #print('pred candidate: ', predictions[i])
            if intersection(annotation, predictions[i]) > 0: #convert_rect_to_point only for tests
                IOU = intersection(annotation, predictions[i])/union(annotation, predictions[i])
                #print(IOU)  
                if(IOU > max_IOU):
                    max_IOU_index = i
                    max_IOU = IOU
        if max_IOU > IOU_thres:
            # remove prediction from list
            predictions.pop(max_IOU_index)
        else:
            false_neg = false_neg + 1
    
    false_pos = len(predictions)
    true_pos = len(preds) - len(predictions)
    return (false_neg, false_pos, true_pos)        

### Import Data and Model 

In [3]:
# import data
file = open(file_location)
file_paths = list(file)  

print(file_paths)

# import model
model_path = os.path.join('..','..','Create-CSV-dataset', 'snapshots', 'resnet50_csv_25.h5');

model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model)


labels_to_names = {0: 'Brick'};

#labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

['colorIMG_15.png\n', 'colorIMG_11.png\n', 'colorIMG_19.png\n', 'colorIMG_101.png\n', 'colorIMG_75.png\n', 'colorIMG_30.png\n', 'colorIMG_178.png\n', 'colorIMG_100.png\n', 'colorIMG_185.png\n', 'colorIMG_79.png\n', 'colorIMG_52.png\n', 'colorIMG_182.png\n', 'colorIMG_42.png\n', 'colorIMG_189.png\n', 'colorIMG_162.png\n', 'colorIMG_12.png\n', 'colorIMG_27.png\n', 'colorIMG_145.png\n', 'colorIMG_96.png\n', 'colorIMG_107.png\n', 'colorIMG_194.png\n', 'colorIMG_60.png\n', 'colorIMG_128.png\n', 'colorIMG_86.png\n', 'colorIMG_40.png\n', 'colorIMG_105.png\n', 'colorIMG_36.png\n', 'colorIMG_62.png\n', 'colorIMG_133.png\n', 'colorIMG_6.png\n', 'colorIMG_103.png\n', 'colorIMG_70.png\n', 'colorIMG_135.png\n', 'colorIMG_61.png\n', 'colorIMG_26.png\n', 'colorIMG_193.png\n', 'colorIMG_146.png\n', 'colorIMG_35.png\n', 'colorIMG_138.png\n', 'colorIMG_142.png\n']
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50

### Infer Images

In [ ]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    prediction = model.predict_on_batch(np.expand_dims(image, axis=0))
    print(prediction)

In [ ]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0
alpha = 0.5

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    draw2 = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels, angles = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    # print detections
    for box, score, label, angles in zip(boxes[0], scores[0], labels[0], angles[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw2, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        draw_vector(draw, angles[0], angles[1], box)
        draw_caption(draw2, b, caption)
        draw_vector(draw2, angles[0], angles[1], box)
    
    cv2.addWeighted(draw, alpha, draw2, 1 - alpha,0, draw)
    # load json file containing annotations
    annotations = read_annotations_from_json(path.strip('\n'))
    
    # remove bounding boxes with a classification score below accept_BB_threshold. 
    predictions = filter_bounding_boxes(boxes[0], scores[0])
    
    #calculate intersection and union of two bounding boxes
    (false_neg, false_pos, true_pos) = evaluate_predictions(annotations, predictions, IOU_threshold)
    #results = evaluate_predictions(annotations, annotations)
    total_false_negative = total_false_negative + false_neg
    total_false_positive = total_false_positive + false_pos
    total_true_positive  = total_true_positive  + true_pos
    
    # store image
    cv2.imwrite(place_to_store_results + path.strip('\n'), draw)

# Calculate precision, recall and F1 score 
precision = total_true_positive / (total_true_positive + total_false_positive)
recall = total_true_positive / (total_true_positive + total_false_negative)
f1 = 2 * (precision * recall) / (precision + recall) 

# print result
print('Precision: ', precision)
print('Recall: ', recall)
print('f1: ', f1)